Build model of energy metabolism described in Faizi et al 2018.

In [9]:
import tellurium as te
import math
import numpy as np
import matplotlib.pyplot as plt

In [14]:
model energy_metabolism
    compartment Cell;
    Cell = 1.0;
    species C_extracellular, C_intracellular, Energy, AminoAcid, Transporter, Metaenzyme, Photoenzyme, Ribosome;
    C_extracellular in cell;
    C_intracellular in cell;
    Energy in cell;
    AminoAcid in cell;
    Transporter in cell;
    Metaenzyme in cell;
    Photoenzyme in cell; 
    Ribosome in cell; 
    # var cix; # extracellular inorganic carbon
    # var ci; # intracellular inorganic carbon
    # var e; # energy units (here representing the ATP, NADH, etc pool)
    # var aa; # amino acids
    # var P; # inactivated and activated photosynthetic machinery
    # var T; # transporter protein for inorganic carbon
    # var M; # metabolic multi-enzyme complex
    # var R; # ribosome
    
    Pm = 0.0018; # permeability, dm min-1
    a_cell = 3.85e-9; # cell surface area, dm2 cell-1
    v_cell = 2.24e-14; # L, cell-1
    N_a = 6.022e23; # Avogadro constant, mol-1
    k_cat_T = 726; # maximal import rate, min-1
    K_T = 15; # half-sat constant of T, uM
    k_cat_M = 545; # maximal metabolic rate, min-1
    K_M = 2441560; # half-sat constant of M, molecules cell-1
    Y_max = 1320; # maximal translation rate, aa min-1 molecules-1
    K_aa = 10000; # half-sat constant for aa, molecule cell-1
    K_e = 10000; # half-sat constant for e, molecule cell-1
    abs_xsec = 0.166; # effective absorption cross-section of P, m2 uE-1
    turnover_P = 1900; # maximal turnover rate of P, min-1
    k_d = 1.6e-6; # rate constant for photodamage
    K_i = 1.0e8; # product-inhibition constant for e synthesis, molecule cell-1
    dens = 1.4e10; # average density, aa cell-1
    m_u = 45; # e needed to create one aa
    m_y = 3; # e needed for one translational elongation step
    m_omega = 8; # e produced by P cycle
    m_c = 5; # average carbon chain length of aa
    D_c = 1.4e10 # aa cell^-1
    sigma = 1; # ?
    
    C_extracellular = 1000;
    C_intracellular = 1; 
    Ribosome = 1.0e5;
    Transporter = 1.0e4;
    Metaenzyme = 1.0e5;
    Photoenzyme = 1.0e5;
    Energy = 1.0e7;
    AminoAcid = 1.0e6;
    Irradiance = 1000;
    
    gamma_max = 1320 #aa molecule^-1 min^-1
    K_E = 10000 # molecule cell^-1
    K_AA = 10000 # molecule cell^-1
    K_2 = 1;
    
    beta_R = 0.25;
    beta_T = 0.25;
    beta_M = 0.25;
    beta_P = 0.25;
    beta_J = 1;
    
    n_R = 7358; # aa molecule^-1
    n_T = 1681; # aa molecule^-1
    n_M = 28630; # aa molecule^-1
    n_P = 95451; # aa molecule^-1
    
    v_t := Transporter * k_cat_T * (C_extracellular/(K_T+C_extracellular)) * (Energy/(K_e+Energy)) # transport rate
    v_m := Metaenzyme + k_cat_M * (C_intracellular/(K_M+C_intracellular)) * (Energy/(K_e+Energy)) # assimilation rate
    mu := (1/D_c) * (v_t/m_c) # growth rate
    gamma_j := beta_J * gamma_max/(n_R + n_T + n_M +n_P) * Ribosome * (Energy/(K_e+Energy)) * (AminoAcid/(K_aa+AminoAcid))
    v_2 := (K_2 * Energy * sigma * Irradiance * Photoenzyme)/(sigma * Irradiance + K_2 * Energy + mu)
    
    Energy_P: => Energy; m_omega * v_2 # energy produced
    Energy_C: Energy =>; v_t + m_u * v_m + mu * Energy + m_y * (beta_R*n_R + beta_T*n_T + beta_M*n_M + beta_P*n_P) # energy consumed
    
    end 
    

Success: Model can be accessed via variable energy_metabolism


In [16]:
energy_metabolism.reset()
energy_metabolism.simulate(start = 0., end = 1.0e3, steps = 1000, selections=['time', 'Energy_P', 'Energy_C']) 
energy_metabolism.plot()